In [1]:
import numpy as np
import pandas as pd

import urllib.request
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier     # obtiene un score (test/pred/submin): 0.3455/0.2211/0.23092
from sklearn.linear_model import LogisticRegression     # obtiene un score (test/pred/submin): 0.277/0.2717/0.27284 ##  de momento es el mejor modelo con 0'27
from sklearn.svm import SVC                             # se para porque no arranca
from catboost import CatBoostClassifier                 # obtiene un score (test/pred/submin): 0.3623/0.2230/0.23423
from sklearn.ensemble import GradientBoostingClassifier # obtiene un score (test/pred/submin): 0.3661/0.222/
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, RepeatedKFold # RepeatedStratifiedKFold sólo se utiliza para categorías 

In [19]:
df_cols = pd.read_csv('columns_meaning.csv')
df_hospital_train = pd.read_csv("hospital_train.csv")
df_hospital_train = df_hospital_train.dropna()
#df_hospital_train['17'].value_counts('21-30')  -> toma este valor de referencia ya que es el más frecuente, pero no ajusta nada -> no sirve

In [4]:
# limpieza del dataframe original y codificación de los datos
# definición de las funciones de codificación
def col12_toInt (strin):
    if strin == 'Urgent':
        return 100
    elif strin == 'Trauma':
        return 10
    elif strin == 'Emergency':
        return 1

def col13_toInt (strin):
    if strin == 'Extreme':
        return 100
    elif strin == 'Moderate':
        return 10
    elif strin == 'Minor':
        return 1

def col2_toInt (strin):
    if strin == 'a':
        return 44130
    elif strin == 'b':
        return 21667
    elif strin == 'c':
        return 14145
    elif strin == 'd':
        return 6341
    elif strin == 'e':
        return 7678
    elif strin == 'f':
        return 3237
    elif strin == 'g':
        return 1286

In [5]:
# selección de las columnas
encod_df_hospital_train = df_hospital_train[['2','9','11','12','13','14','15','16']] # incluyendo col 2 -> score 

# ejecucción de las funciones de codificación
encod_df_hospital_train['2'] = encod_df_hospital_train['2'].apply(lambda x: col2_toInt(x))
encod_df_hospital_train['12'] = encod_df_hospital_train['12'].apply(lambda x: col12_toInt(x))
encod_df_hospital_train['13'] = encod_df_hospital_train['13'].apply(lambda x: col13_toInt(x))
encod_df_hospital_train['15'] = encod_df_hospital_train['15'].apply(lambda x: int(x.replace('-', '')))

X = np.array(encod_df_hospital_train)
y = np.array(df_hospital_train['17'])
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle= True, test_size = 0.50, random_state = 42)



ipykernel_launcher:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:8: SettingWithCopyWarning: 
A value is trying to be

In [22]:
rad_st = [42, 51,75,62,355,48,6984,66,354,4899]
models=[]

for elem in rad_st:
    model = LogisticRegression(class_weight= 'balanced', random_state=elem, n_jobs= -1)
    model.fit(X_train,y_train)
    models.append(model)

    # 5. Predecir con el modelo ya entrenado con X_test
    predictions = model.predict(X_test)
    print('predicciones test:', predictions)
    print('valores test:', y_test)
    score = model.score(X_test, y_test)
    print(elem, score)

predicciones test: ['11-20' 'More than 100 Days' '11-20' ... '0-10' '41-50' '11-20']
valores test: ['31-40' '21-30' '21-30' ... '21-30' '21-30' '21-30']
42 0.1551114901913001
predicciones test: ['11-20' 'More than 100 Days' '11-20' ... '0-10' '41-50' '11-20']
valores test: ['31-40' '21-30' '21-30' ... '21-30' '21-30' '21-30']
51 0.1551114901913001
predicciones test: ['11-20' 'More than 100 Days' '11-20' ... '0-10' '41-50' '11-20']
valores test: ['31-40' '21-30' '21-30' ... '21-30' '21-30' '21-30']
75 0.1551114901913001
predicciones test: ['11-20' 'More than 100 Days' '11-20' ... '0-10' '41-50' '11-20']
valores test: ['31-40' '21-30' '21-30' ... '21-30' '21-30' '21-30']
62 0.1551114901913001
predicciones test: ['11-20' 'More than 100 Days' '11-20' ... '0-10' '41-50' '11-20']
valores test: ['31-40' '21-30' '21-30' ... '21-30' '21-30' '21-30']
355 0.1551114901913001
predicciones test: ['11-20' 'More than 100 Days' '11-20' ... '0-10' '41-50' '11-20']
valores test: ['31-40' '21-30' '21-30' 

In [27]:
model = LogisticRegression(random_state=42, n_jobs= -1, class_weight='balanced')
model.fit(X_train,y_train)
models.append(model)

# 5. Predecir con el modelo ya entrenado con X_test
predictions = model.predict(X_test)
print('predicciones test:', predictions)
print('valores test:', y_test)
score = model.score(X_test, y_test)
print(elem, score)

predicciones test: ['11-20' 'More than 100 Days' '11-20' ... '0-10' '41-50' '11-20']
valores test: ['31-40' '21-30' '21-30' ... '21-30' '21-30' '21-30']
4899 0.1551114901913001


In [21]:
df = pd.DataFrame(list(predictions), index='0')
df

TypeError: Index(...) must be called with a collection of some kind, '0' was passed

In [15]:
model.classes_

array(['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70',
       '71-80', '81-90', '91-100', 'More than 100 Days'], dtype=object)

In [23]:
# Carga de los datos
df_hospital_test = pd.read_csv("hospital_test.csv")
sample = pd.read_csv("sample_submission.csv")

# selección de las columnas
encod_df_hospital_test = df_hospital_test[['2','9','11','12','13','14','15','16']]

# ejecucción de las funciones de codificación
encod_df_hospital_test['2'] = encod_df_hospital_test['2'].apply(lambda x: col2_toInt(x))
encod_df_hospital_test['12'] = encod_df_hospital_test['12'].apply(lambda x: col12_toInt(x))
encod_df_hospital_test['13'] = encod_df_hospital_test['13'].apply(lambda x: col13_toInt(x))
encod_df_hospital_test['15'] = encod_df_hospital_test['15'].apply(lambda x: int(x.replace('-', '')))

# sustitución de los valores nan por valores reales (lo ideal sería hacerlo de forma automática)
encod_df_hospital_test['11'].fillna(8, inplace= True) # 8 es el valor que más se repite tanto en el conjunto de train como en el predicción
encod_df_hospital_test['9'].fillna(2, inplace= True) # 2 es el valor que más se repite tanto en el conjunto de train como en el predicción

# se convierten los datos a vectores y se ejecutan las predicciones con el modelo entrenado
X_pred = np.array(encod_df_hospital_test)
y_sample = sample['days']
predictions_submit = model.predict(X_pred)
submission = pd.DataFrame({"id": df_hospital_test['0'], "days": predictions_submit})

ipykernel_launcher:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:12: SettingWithCopyWarning: 
A value is trying to

In [15]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")
chequeator(submission)

You're ready to submit!


In [17]:
model.score(X_pred, y_sample)

0.27173555449216413

In [147]:
list11 = list(encod_df_hospital_test['11'].unique())
code11 = []
count11 = []
for elem in list11:
    code11.append(elem)
    count11.append(list(encod_df_hospital_test['11']).count(elem))
data11 = pd.DataFrame(list(zip(code11, count11)), columns=['code', 'data'])
data11.sort_values('data', ascending=False)
data11.sum()

code       724.0
data    131927.0
dtype: float64

In [41]:
list11 = list(encod_df_hospital_test['9'].unique())
code11 = []
count11 = []
for elem in list11:
    code11.append(elem)
    count11.append(list(encod_df_hospital_test['9']).count(elem))
data11 = pd.DataFrame(list(zip(code11, count11)), columns=['code', 'data'])
data11.sort_values('data', ascending=False)
data11.sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of    code   data
0   2.0  52019
1   4.0  23941
2   1.0  11295
3   3.0  46449
4   NaN      0>

In [ ]:
list11 = list(encod_df_hospital_train['11'].unique())
code11 = []
count11 = []
for elem in list11:
    code11.append(elem)
    count11.append(list(encod_df_hospital_train['11']).count(elem))
data11 = pd.DataFrame(list(zip(code11, count11)), columns=['code', 'data'])
data11.sort_values('data', ascending=False)

In [26]:
sample['days'].value_counts()

21-30                 36744
11-20                 32762
31-40                 23145
51-60                 14742
0-10                  10062
41-50                  4933
71-80                  4253
More than 100 Days     2769
81-90                  2077
91-100                 1138
61-70                  1119
Name: days, dtype: int64